### Get Gene Info

In [1]:
import pandas as pd
from Bio import Entrez
import time
import Request_Utilis

Entrez.email = "a.antonakoudis@sartorius.com"
excel_file = '../Input/masterTable.csv'

genes = pd.read_csv(excel_file)


In [ ]:
# The entrezID for human is correct for everything, so use that as input
# Get the EnsembleID, the one in the top page of the NCBI
# 1 mRNA to 1 ProteinID
# Get UniprotID
# Use the shared exchel from Pablo


In [6]:
for gene_EntrezID in genes['ENTREZID'][0:2]:
    
    protein_sequence_ID_list = []
    transcript_sequence_ID_list = []

    # search_results = Entrez.esearch(db="gene", term=f"{gene_name}[Gene Name] AND {organism}[Organism]")
    # record = Entrez.read(search_results)
    gene_record = Request_Utilis.read_gene_ID(int(gene_EntrezID))

    # for gene_name_EntrezID in record['IdList'][0:]:
    #     gene_info = Entrez.efetch(db="gene", id=gene_name_EntrezID, retmode="xml") 
    #     gene_record = Entrez.read(gene_info)

    org = gene_record[0]['Entrezgene_source']['BioSource']['BioSource_org']['Org-ref']['Org-ref_taxname']
    if 'Gene-ref_syn' in gene_record[0]['Entrezgene_gene']['Gene-ref']:
        gene_synonyms = gene_record[0]['Entrezgene_gene']['Gene-ref']['Gene-ref_syn']
    else:
        gene_synonyms = []

    # Get the gene name and gene NCBI ID
    if 'Gene-ref_locus' in gene_record[0]['Entrezgene_gene']['Gene-ref']:
        gene_symbol = gene_record[0]['Entrezgene_gene']['Gene-ref']['Gene-ref_locus']                        
    else:
        gene_symbol = gene_record[0]['Entrezgene_gene']['Gene-ref']['Gene-ref_locus-tag']
        
    # Get the Gene description
    if 'Gene-ref_desc' in gene_record[0]['Entrezgene_gene']['Gene-ref']:
        gene_name = gene_record[0]['Entrezgene_gene']['Gene-ref']['Gene-ref_desc']
    else:
        gene_name = gene_record[0]['Entrezgene_locus'][0]['Gene-commentary_products'][0]['Gene-commentary_label']
        
    # Get Human and Mouse Orthologs
    mouse_ortholog_EntrezID = Request_Utilis.get_gene_ids(gene_EntrezID, '10090')
    cho_ortholog_EntrezID = Request_Utilis.get_gene_ids(gene_EntrezID, '10029')

    print("Organism: ", org, "|Entrez ID: ", gene_EntrezID, "|Gene Symbol: ", gene_symbol, "|Gene Name: ", gene_name, "|Gene Synonyms: ", gene_synonyms, "| Mouse: ", mouse_ortholog_EntrezID, "|CHO:", cho_ortholog_EntrezID)
    for assembly_specific_info in gene_record[0]['Entrezgene_locus']:  
        if 'Gene-commentary_heading' in assembly_specific_info:
            for assembly_specific_transcript in assembly_specific_info['Gene-commentary_products']:
                if 'Gene-commentary_products' in assembly_specific_transcript:
                    transcript_sequence_ID = assembly_specific_transcript['Gene-commentary_accession']

                    print("|mRNA: ", transcript_sequence_ID)
                    transcript_sequence_ID_list.append(transcript_sequence_ID)

                    for Entrez_Comments in gene_record[0]['Entrezgene_comments']:
                        if 'Gene-commentary_comment' in Entrez_Comments:
                            for comments in Entrez_Comments['Gene-commentary_comment']:
                                if 'Gene-commentary_products' in comments:
                                    for product_per_assembly in comments['Gene-commentary_products']:                                
                                        if product_per_assembly['Gene-commentary_heading'] == 'mRNA Sequence':
                                            mRNA = product_per_assembly['Gene-commentary_accession']
                                            if mRNA == transcript_sequence_ID:  
                                                for protein_per_transcript_per_assemlby in product_per_assembly['Gene-commentary_products']:
                                                    protein_sequence_ID = protein_per_transcript_per_assemlby['Gene-commentary_accession']

                                                    uniprotID_list = []
                                                    for protein_comments in protein_per_transcript_per_assemlby['Gene-commentary_comment']:
                                                        if protein_comments['Gene-commentary_heading'] == 'UniProtKB':
                                                            for uniprotID_protein_per_assembly in protein_comments['Gene-commentary_comment'][0]['Gene-commentary_source']:
                                                                protein_uniprot_id = uniprotID_protein_per_assembly['Other-source_src']['Dbtag']['Dbtag_tag']['Object-id']['Object-id_str']
                                                                uniprotID_list.append(protein_uniprot_id)


                                                    print("|Protein: ", protein_sequence_ID, uniprotID_list)
                                                    protein_sequence_ID_list.append(protein_sequence_ID)
                                        

                                        else:   

                                            if 'Gene-commentary_products' in product_per_assembly:
                                                for transcript_per_assembly in product_per_assembly['Gene-commentary_products']:      
                                                    mRNA = transcript_per_assembly['Gene-commentary_accession']          
                                                    if mRNA == transcript_sequence_ID:                                                  
                                                        for protein_per_transcript_per_assemlby in transcript_per_assembly['Gene-commentary_products']:                                                                
                                                            protein_sequence_ID = protein_per_transcript_per_assemlby['Gene-commentary_accession']
                                                            uniprotID_list = []
                                                            if 'Gene-commentary_comment' in protein_per_transcript_per_assemlby:
                                                                for protein_comments in protein_per_transcript_per_assemlby['Gene-commentary_comment']:                                                         
                                                                    if protein_comments['Gene-commentary_heading'] == 'UniProtKB':
                                                                        for uniprotID_protein_per_assembly in protein_comments['Gene-commentary_comment'][0]['Gene-commentary_source']:
                                                                            protein_uniprot_id = uniprotID_protein_per_assembly['Other-source_src']['Dbtag']['Dbtag_tag']['Object-id']['Object-id_str']
                                                                            uniprotID_list.append(protein_uniprot_id)
                                                            else:
                                                                uniprotID_list = []
                                                            print("|Protein: ", protein_sequence_ID, uniprotID_list)
                                                            protein_sequence_ID_list.append(protein_sequence_ID)

    # for index, transcript_ID in enumerate(transcript_sequence_ID_list):
    #     print(transcript_sequence_ID_list[index], " --> ", protein_sequence_ID_list[index])
        

Organism:  Homo sapiens |Entrez ID:  127550 |Gene Symbol:  A3GALT2 |Gene Name:  alpha 1,3-galactosyltransferase 2 |Gene Synonyms:  ['IGB3S', 'IGBS3S', 'A3GALT2P'] | Mouse:  215493 |CHO: 100774283
|mRNA:  NM_001080438
|Protein:  NP_001073907 ['U3KPV4']
|mRNA:  NM_001080438
|Protein:  NP_001073907 ['U3KPV4']
Organism:  Homo sapiens |Entrez ID:  53947 |Gene Symbol:  A4GALT |Gene Name:  alpha 1,4-galactosyltransferase (P1PK blood group) |Gene Synonyms:  ['P1', 'PK', 'Gb3S', 'P(k)', 'P1PK', 'A14GALT', 'A4GALT1'] | Mouse:  239559 |CHO: 100770462
|mRNA:  XM_005261648
|Protein:  XP_005261705 ['A0A0S2Z5J1', 'A9XEA7', 'B0L3P6', 'B2R7C4', 'I6S6F0', 'Q540I5', 'Q540I6', 'Q6RJV9', 'Q6RJW0', 'Q6RJW1', 'Q7Z2I1', 'Q9NPC4', 'Q9P1X5']
|mRNA:  NM_001318038
|Protein:  NP_001304967 ['A0A0S2Z5J1', 'A9XEA7', 'B0L3P6', 'B2R7C4', 'I6S6F0', 'Q540I5', 'Q540I6', 'Q6RJV9', 'Q6RJW0', 'Q6RJW1', 'Q7Z2I1', 'Q9NPC4', 'Q9P1X5']
|mRNA:  XM_047441412
|Protein:  XP_047297368 ['A0A0S2Z5J1', 'A9XEA7', 'B0L3P6', 'B2R7C4', 'I6S